In [6]:
import sqlite3
import pandas as pd
import json
from glob import glob
from os import path

In [9]:
people = ['Katies', 'Bens']
for person in people:
    print()
    print(f"finding {person} songs")
    files = glob(path.join(f"{person}Data", "*.json"))
    totalUniqueSongs = set()
    for file in files:
        with open(f'{file}', encoding="utf-8") as json_data:
            data = json.load(json_data)
            if 'Library' in file:
                spotifyDf = pd.DataFrame(data['tracks'])
                
            else:
                spotifyDf = pd.DataFrame(data)
        # only keep the needed columns for sorting
        
        if 'Library' in file:
            keys = ['track', 'artist']
            
        elif 'Streaming' in file:
            keys = ['trackName', 'artistName']
        else:
            keys = ['master_metadata_track_name', 'master_metadata_album_artist_name']
        sortingDf = spotifyDf[keys]
        sortingDf = sortingDf.drop_duplicates()
        print(f"{len(sortingDf)} unique songs found in {file}")   
        escapedDf = sortingDf.replace("'", "''", regex=True)

        connection = sqlite3.connect('track_metadata.db')
        # for every song in library make a sql query and retrieve the track id
        escapedDf['id'] = escapedDf.apply(lambda x: pd.read_sql_query(f"SELECT track_id FROM songs WHERE title='{x[0]}' AND artist_name='{x[1]}';", connection), axis=1)
        connection.close()
        validId =  set([i.iloc[[0]]['track_id'].values[0] for i in escapedDf['id'] if i.empty == False])
        print(f"{len(validId)} songs found that are also in MDS")
        old_len = len(totalUniqueSongs)
        totalUniqueSongs = totalUniqueSongs.union(validId)
        print(f"{len(totalUniqueSongs) - old_len} new unique songs found in {file}")
    print(f"Total: {len(totalUniqueSongs)} songs found for {person} that are also in the MDS")
    filePaths = [f'/songs/data/{i[2]}/{i[3]}/{i[4]}/{i}.h5' for i in totalUniqueSongs]
    validDf = pd.DataFrame(totalUniqueSongs)
    validDf.to_csv(f"trackPaths/{person}_trackNames.csv", index=False)    
        


finding Katies songs
730 unique songs found in KatiesData/KatieStreamingHistory0.json
70 songs found that are also in MDS
70 new unique songs found in KatiesData/KatieStreamingHistory0.json
676 unique songs found in KatiesData/KatiesStreamingHistory1.json
70 songs found that are also in MDS
18 new unique songs found in KatiesData/KatiesStreamingHistory1.json
1363 unique songs found in KatiesData/Katiesendsong_0.json
126 songs found that are also in MDS
59 new unique songs found in KatiesData/Katiesendsong_0.json
1305 unique songs found in KatiesData/Katiesendsong_1.json
128 songs found that are also in MDS
41 new unique songs found in KatiesData/Katiesendsong_1.json
1356 unique songs found in KatiesData/Katiesendsong_2.json
135 songs found that are also in MDS
31 new unique songs found in KatiesData/Katiesendsong_2.json
1371 unique songs found in KatiesData/Katiesendsong_3.json
123 songs found that are also in MDS
20 new unique songs found in KatiesData/Katiesendsong_3.json
69 unique 

OSError: Cannot save file into a non-existent directory: 'trackPaths'

In [38]:

validDf = pd.DataFrame(totalUniqueSongs)
validDf.drop_duplicates()
print(f"final number of unique songs {len(validDf)}")
validDf.to_csv(f"ValidSongs/{person}_ValidSongs.csv", index=False)

final number of unique songs 4845
